# MWE for regenie burden

Aim: create the phenotypic file for the MWE made to run regenie_burden

## 1. Create phenotypic file

In [2]:
pheno <- read.table('/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_withagesex_033120', header=T)
head(pheno)

,FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1000019,1000019,47,0,1,0,32.4868,96,110
2,1000022,1000022,53,1,0,0,28.5292,102,111
3,1000035,1000035,63,1,0,0,26.1551,94,102
4,1000046,1000046,62,0,0,0,29.4740,93,98
5,1000054,1000054,65,0,0,0,30.2294,103,109
6,1000063,1000063,43,1,0,0,24.8432,83,100


In [4]:
# Exome data
pheno_exome <- read.table('/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_s200631_188448ind.fam', header=T)
head(pheno_exome)

,FID,IID,ignore1,ignore2,ignore3,ignore4
,<int>,<int>,<int>,<int>,<int>,<int>
1,1434748,1434748,0,0,2,-9
2,5523981,5523981,0,0,1,-9
3,5023838,5023838,0,0,2,-9
4,4023729,4023729,0,0,1,-9
5,4515669,4515669,0,0,2,-9
6,1129683,1129683,0,0,1,-9


In [5]:
shared_IID <- pheno_exome[pheno_exome$IID %in% pheno$IID,]
dim(shared_IID)
head(shared_IID)

[1] 146904      6

,FID,IID,ignore1,ignore2,ignore3,ignore4
,<int>,<int>,<int>,<int>,<int>,<int>
1,1434748,1434748,0,0,2,-9
2,5523981,5523981,0,0,1,-9
3,5023838,5023838,0,0,2,-9
4,4023729,4023729,0,0,1,-9
5,4515669,4515669,0,0,2,-9
8,4744741,4744741,0,0,1,-9


In [6]:
filter_IID <- pheno[pheno$IID %in% shared_IID$IID,]
dim(filter_IID)
head(filter_IID)

[1] 146904      9

,FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1000019,1000019,47,0,1,0,32.4868,96,110
3,1000035,1000035,63,1,0,0,26.1551,94,102
7,1000078,1000078,52,0,0,0,26.3960,80,109
16,1000198,1000198,41,0,0,0,34.5247,101,115
18,1000210,1000210,66,1,0,0,28.2402,104,110
19,1000224,1000224,58,0,0,0,22.2151,75,95


In [7]:
library(dplyr)
filter_cases <- filter_IID %>%
  mutate(case = case_when(ASTHMA == 1 & T2D==1 ~ 2),
         SEX = case_when(SEX == 0 ~ 1, SEX == 1 ~ 2)) %>%
  filter(case==2) %>%
  slice_head(n=100)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [21]:
head(filter_cases)
dim(filter_cases)

,FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP,case
,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1000516,1000516,46,2,1,1,20.6489,76,87,2
2,1002888,1002888,68,2,1,1,25.0142,94,96,2
3,1004336,1004336,67,2,1,1,36.2103,126,115,2
4,1006972,1006972,51,2,1,1,32.6235,111,110,2
5,1014688,1014688,49,2,1,1,27.4902,92,100,2
6,1015639,1015639,65,2,1,1,25.7300,97,92,2


[1] 100  10

In [22]:
filter_cases %>% 
    filter(FID < 1)


FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP,case
<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>


In [9]:
filter_ctrl <- filter_IID %>%
  mutate(case = case_when(ASTHMA == 0 & T2D==0 ~ 1),
          SEX = case_when(SEX == 0 ~ 1, SEX == 1 ~ 2)) %>%
  filter(case==1) %>%
  slice_head(n=100)

In [10]:
dim(filter_ctrl)
head(filter_ctrl)

[1] 100  10

,FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP,case
,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1000035,1000035,63,2,0,0,26.1551,94,102,1
2,1000078,1000078,52,1,0,0,26.3960,80,109,1
3,1000198,1000198,41,1,0,0,34.5247,101,115,1
4,1000210,1000210,66,2,0,0,28.2402,104,110,1
5,1000224,1000224,58,1,0,0,22.2151,75,95,1
6,1000236,1000236,61,2,0,0,29.4934,108,110,1


In [11]:
phenotype <- bind_rows(filter_cases, filter_ctrl)
phenotype <- phenotype[ -c(10) ]

In [12]:
head(phenotype)
dim(phenotype)

,FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP
,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1000516,1000516,46,2,1,1,20.6489,76,87
2,1002888,1002888,68,2,1,1,25.0142,94,96
3,1004336,1004336,67,2,1,1,36.2103,126,115
4,1006972,1006972,51,2,1,1,32.6235,111,110
5,1014688,1014688,49,2,1,1,27.4902,92,100
6,1015639,1015639,65,2,1,1,25.7300,97,92


[1] 200   9

In [23]:
write.table(phenotype,'/gpfs/gibbs/pi/dewan/data/UKBiobank/MWE/burden/phenotype_burden.txt', sep="\t", row.names=FALSE, col.names=T, quote=F)

In [14]:
phenotype_IID <- phenotype[ -c(3:10) ]
head(phenotype_IID)

,FID,IID
,<int>,<int>
1,1000516,1000516
2,1002888,1002888
3,1004336,1004336
4,1006972,1006972
5,1014688,1014688
6,1015639,1015639


In [24]:
write.table(phenotype_IID,'/gpfs/gibbs/pi/dewan/data/UKBiobank/MWE/burden/phenotype_burden_IID.txt', sep="\t", row.names=FALSE, col.names=F, quote=F)

In [16]:
unrel <- read.table('/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620', header=T)

In [17]:
head(unrel)
dim(unrel)

,FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP,WAISTHIPRATIO
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1000019,1000019,47,0,1,0,32.4868,96,110,0.872727
2,1000022,1000022,53,1,0,0,28.5292,102,111,0.918919
3,1000035,1000035,63,1,0,0,26.1551,94,102,0.921569
4,1000046,1000046,62,0,0,0,29.4740,93,98,0.948980
5,1000054,1000054,65,0,0,0,30.2294,103,109,0.944954
6,1000063,1000063,43,1,0,0,24.8432,83,100,0.830000


[1] 301931     10

In [18]:
unrelated_shared <- phenotype[phenotype$IID %in% unrel$IID,]
dim(unrelated_shared)
head(unrelated_shared)

[1] 168   9

,FID,IID,AGE,SEX,ASTHMA,T2D,BMI,WAIST,HIP
,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1000516,1000516,46,2,1,1,20.6489,76,87
2,1002888,1002888,68,2,1,1,25.0142,94,96
4,1006972,1006972,51,2,1,1,32.6235,111,110
5,1014688,1014688,49,2,1,1,27.4902,92,100
8,1017209,1017209,67,1,1,1,31.5188,94,114
10,1026242,1026242,63,1,1,1,36.8052,106,117


In [19]:
write.table(unrelated_shared,'/gpfs/gibbs/pi/dewan/data/UKBiobank/MWE/burden/unrelated_ind_burden.txt', sep="\t", row.names=FALSE, col.names=T, quote=F)

## 2. Create annotation file

In [94]:
annotation <- read.csv("/home/dc2325/scratch60/output/bfile_annovar/010421_UKBB_Hearing_aid_f3393_128254ind_exomes_hearing_aid_cat.regenie.hg38_multianno.csv")
head(annotation)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,⋯,CLNDN,CLNDISDB,CLNREVSTAT,CLNSIG,DN.ID,Patient.ID,Phenotype,Platform,Study,Pubmed.ID
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1,33010786,33010786,C,T,UTR3,AK2,NM_001319139:c.*43G>A;NM_001199199:c.*43G>A;NM_001319141:c.*92G>A;NM_013411:c.*43G>A;NM_001319142:c.*43G>A;NM_001625:c.*2395G>A,.,.,⋯,.,.,.,.,.,.,.,.,.,.
2,1,58468867,58468867,T,C,intergenic,DAB1;OMA1,dist=218320;dist=11852,.,.,⋯,.,.,.,.,.,.,.,.,.,.
3,1,58468874,58468874,T,C,intergenic,DAB1;OMA1,dist=218327;dist=11845,.,.,⋯,.,.,.,.,.,.,.,.,.,.
4,1,58468970,58468970,G,T,intergenic,DAB1;OMA1,dist=218423;dist=11749,.,.,⋯,.,.,.,.,.,.,.,.,.,.
5,1,58506159,58506159,G,A,exonic,OMA1,.,synonymous SNV,OMA1:NM_145243:exon8:c.C1266T:p.F422F,⋯,.,.,.,.,.,.,.,.,.,.
6,1,58506268,58506268,C,A,intronic,OMA1,.,.,.,⋯,.,.,.,.,.,.,.,.,.,.


In [1]:
anno_exome <- read.csv("/gpfs/gibbs/pi/dewan/data/UKBiobank/results/annovar_exome/ukb23155_chr1_chr22_exomedata.hg38.hg38_multianno.csv")
head(anno_exome)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,⋯,CLNDN,CLNDISDB,CLNREVSTAT,CLNSIG,DN.ID,Patient.ID,Phenotype,Platform,Study,Pubmed.ID
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1,69081,69081,G,C,upstream,OR4F5,dist=10,.,.,⋯,.,.,.,.,.,.,.,.,.,.
2,1,69134,69134,A,G,exonic,OR4F5,.,nonsynonymous SNV,OR4F5:NM_001005484:exon1:c.A44G:p.E15G,⋯,.,.,.,.,.,.,.,.,.,.
3,1,69149,69149,T,A,exonic,OR4F5,.,nonsynonymous SNV,OR4F5:NM_001005484:exon1:c.T59A:p.L20Q,⋯,.,.,.,.,.,.,.,.,.,.
4,1,69217,69217,G,A,exonic,OR4F5,.,nonsynonymous SNV,OR4F5:NM_001005484:exon1:c.G127A:p.V43I,⋯,.,.,.,.,.,.,.,.,.,.
5,1,69224,69224,A,T,exonic,OR4F5,.,nonsynonymous SNV,OR4F5:NM_001005484:exon1:c.A134T:p.D45V,⋯,.,.,.,.,.,.,.,.,.,.
6,1,69231,69231,C,T,exonic,OR4F5,.,synonymous SNV,OR4F5:NM_001005484:exon1:c.C141T:p.H47H,⋯,.,.,.,.,.,.,.,.,.,.


In [2]:
library(tidyverse)
anno_file <- anno_exome %>%
  select(Chr, Start, End, Ref, Alt, Gene.refGene,ExonicFunc.refGene)
head(anno_file)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.5     ✔ dplyr   1.0.3
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



,Chr,Start,End,Ref,Alt,Gene.refGene,ExonicFunc.refGene
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,1,69081,69081,G,C,OR4F5,.
2,1,69134,69134,A,G,OR4F5,nonsynonymous SNV
3,1,69149,69149,T,A,OR4F5,nonsynonymous SNV
4,1,69217,69217,G,A,OR4F5,nonsynonymous SNV
5,1,69224,69224,A,T,OR4F5,nonsynonymous SNV
6,1,69231,69231,C,T,OR4F5,synonymous SNV


In [3]:
anno_file$var_id <- paste0(anno_file$Chr,":",anno_file$Start,":",anno_file$Ref,":",anno_file$Alt)

In [4]:
head(anno_file, 100)

,Chr,Start,End,Ref,Alt,Gene.refGene,ExonicFunc.refGene,var_id
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<chr>
1,1,69081,69081,G,C,OR4F5,.,1:69081:G:C
2,1,69134,69134,A,G,OR4F5,nonsynonymous SNV,1:69134:A:G
3,1,69149,69149,T,A,OR4F5,nonsynonymous SNV,1:69149:T:A
4,1,69217,69217,G,A,OR4F5,nonsynonymous SNV,1:69217:G:A
5,1,69224,69224,A,T,OR4F5,nonsynonymous SNV,1:69224:A:T
6,1,69231,69231,C,T,OR4F5,synonymous SNV,1:69231:C:T
7,1,69270,69270,A,G,OR4F5,synonymous SNV,1:69270:A:G
8,1,69281,69281,T,C,OR4F5,nonsynonymous SNV,1:69281:T:C
9,1,69314,69314,T,C,OR4F5,nonsynonymous SNV,1:69314:T:C


In [5]:
unique(anno_file$ExonicFunc.refGene)

[1] .                          nonsynonymous SNV         
[3] synonymous SNV             stopgain                  
[5] nonframeshift substitution frameshift substitution   
[7] stoploss                   startloss                 
[9] unknown                   
9 Levels: . frameshift substitution ... unknown

In [29]:
recode <- anno_file %>% 
    mutate(annotation = case_when(
       ExonicFunc.refGene == "nonsynonymous SNV" ~ "missense",
       ExonicFunc.refGene %in% c("frameshift substitution", "stoploss","stopgain", "startloss") ~ "LoF",
       ExonicFunc.refGene == "synonymous SNV" ~ "synonymous")) %>%
    filter(Chr==22)

In [30]:
head(recode, 100)

,Chr,Start,End,Ref,Alt,Gene.refGene,ExonicFunc.refGene,var_id,annotation
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>
1,22,15528096,15528096,G,A,OR11H1,.,22:15528096:G:A,NA
2,22,15528109,15528109,T,G,OR11H1,.,22:15528109:T:G,NA
3,22,15528133,15528133,A,G,OR11H1,.,22:15528133:A:G,NA
4,22,15528155,15528155,G,A,OR11H1,.,22:15528155:G:A,NA
5,22,15528165,15528165,C,A,OR11H1,nonsynonymous SNV,22:15528165:C:A,missense
6,22,15528166,15528166,C,T,OR11H1,nonsynonymous SNV,22:15528166:C:T,missense
7,22,15528177,15528177,C,T,OR11H1,stopgain,22:15528177:C:T,LoF
8,22,15528179,15528179,G,T,OR11H1,nonsynonymous SNV,22:15528179:G:T,missense
9,22,15528182,15528182,C,T,OR11H1,synonymous SNV,22:15528182:C:T,synonymous


In [32]:
ann_regie <- recode %>%
    select(var_id, Gene.refGene, annotation)

In [33]:
head(ann_regie)

,var_id,Gene.refGene,annotation
,<chr>,<fct>,<chr>
1,22:15528096:G:A,OR11H1,NA
2,22:15528109:T:G,OR11H1,NA
3,22:15528133:A:G,OR11H1,NA
4,22:15528155:G:A,OR11H1,NA
5,22:15528165:C:A,OR11H1,missense
6,22:15528166:C:T,OR11H1,missense


In [36]:
write.table(ann_regie, "/gpfs/gibbs/pi/dewan/data/UKBiobank/MWE/burden/annotation_file.txt",sep=" ", row.names=FALSE, col.names=F, quote=F)

In [25]:
set_regie <- recode %>%
    select(Gene.refGene,Chr,Start,var_id, annotation) %>%
    filter(Chr==22)
head(set_regie)

,Gene.refGene,Chr,Start,var_id,annotation
,<fct>,<int>,<int>,<chr>,<chr>
1,OR11H1,22,15528096,22:15528096:G:A,NA
2,OR11H1,22,15528109,22:15528109:T:G,NA
3,OR11H1,22,15528133,22:15528133:A:G,NA
4,OR11H1,22,15528155,22:15528155:G:A,NA
5,OR11H1,22,15528165,22:15528165:C:A,missense
6,OR11H1,22,15528166,22:15528166:C:T,missense


In [26]:
unique(set_regie$Gene.refGene)

[1] OR11H1                                              
   [2] POTEH                                               
   [3] POTEH;POTEH                                         
   [4] POTEH-AS1                                           
   [5] DUXAP8;CCT8L2                                       
   [6] CCT8L2                                              
   [7] XKR3                                                
   [8] XKR3;XKR3                                           
   [9] GAB4                                                
  [10] GAB4;GAB4                                           
  [11] IL17RA                                              
  [12] IL17RA;IL17RA                                       
  [13] TMEM121B                                            
  [14] TMEM121B;TMEM121B                                   
  [15] HDHD5                                               
  [16] HDHD5;HDHD5                                         
  [17] ADA2                                                
  [18] ADA2;ADA2                                           
  [19] CECR2                                               
  [20] CECR2;CECR2                                         
  [21] SLC25A18                                            
  [22] SLC25A18;SLC25A18                                   
  [23] LOC101929372;SLC25A18                               
  [24] LOC101929372                                        
  [25] LOC101929372;LOC101929372                           
  [26] LOC101929372;SLC25A18;SLC25A18                      
  [27] ATP6V1E1                                            
  [28] ATP6V1E1;ATP6V1E1                                   
  [29] BCL2L13                                             
  [30] BCL2L13;BCL2L13                                     
  [31] BID                                                 
  [32] BID;BID                                             
  [33] MICAL3                                              
  [34] MICAL3;MICAL3                                       
  [35] PEX26                                               
  [36] PEX26;PEX26                                         
  [37] TUBA8                                               
  [38] TUBA8;TUBA8                                         
  [39] USP18                                               
  [40] USP18;USP18                                         
  [41] TMEM191B                                            
  [42] TMEM191B;TMEM191B                                   
  [43] RIMBP3                                              
  [44] DGCR6;LOC102724770;DGCR6;LOC102724770               
  [45] DGCR6;LOC102724770                                  
  [46] LOC102724788;PRODH                                  
  [47] LOC102724788;PRODH;LOC102724788;PRODH               
  [48] DGCR2                                               
  [49] DGCR2;DGCR2                                         
  [50] DGCR11                                              
  [51] TSSK2;ESS2                                          
  [52] TSSK2                                               
  [53] ESS2;TSSK2                                          
  [54] ESS2                                                
  [55] ESS2;ESS2                                           
  [56] GSC2                                                
  [57] GSC2;GSC2                                           
  [58] SLC25A1                                             
  [59] SLC25A1;SLC25A1                                     
  [60] SLC25A1;CLTCL1                                      
  [61] CLTCL1                                              
  [62] CLTCL1;CLTCL1                                       
  [63] HIRA                                                
  [64] HIRA;HIRA                                           
  [65] MRPL40                                              
  [66] MRPL40;MRPL40                                       
  [67] C22orf39                            

In [27]:
set_list <- set_regie  %>%
    filter(Chr==22 & Gene.refGene=="GAB4" & annotation=="LoF")
head(set_list, 100)

,Gene.refGene,Chr,Start,var_id,annotation
,<fct>,<int>,<int>,<chr>,<chr>
1,GAB4,22,16962748,22:16962748:CCT:C,LoF
2,GAB4,22,16963776,22:16963776:CGGCGGGG:C,LoF
3,GAB4,22,16964805,22:16964805:G:GT,LoF
4,GAB4,22,16966131,22:16966131:AG:A,LoF
5,GAB4,22,16966169,22:16966169:G:A,LoF
6,GAB4,22,16966249,22:16966249:TG:T,LoF
7,GAB4,22,16970027,22:16970027:CT:C,LoF
8,GAB4,22,16987991,22:16987991:G:A,LoF
9,GAB4,22,16987996,22:16987996:G:T,LoF


In [28]:
y <- paste0(set_list$var_id, sep=",")
y

[1] "22:16962748:CCT:C,"                  "22:16963776:CGGCGGGG:C,"            
 [3] "22:16964805:G:GT,"                   "22:16966131:AG:A,"                  
 [5] "22:16966169:G:A,"                    "22:16966249:TG:T,"                  
 [7] "22:16970027:CT:C,"                   "22:16987991:G:A,"                   
 [9] "22:16987996:G:T,"                    "22:16988055:C:CG,"                  
[11] "22:16988084:G:A,"                    "22:16988117:CTGGAGAAGAGCAGAGGCCG:C,"
[13] "22:16991922:C:T,"                    "22:16992153:GATAAACCA:G,"           
[15] "22:16992162:GC:G,"                   "22:17007961:CG:C,"                  
[17] "22:17007961:C:CG,"                   "22:17007994:GCACGTGGC:G,"           
[19] "22:17008009:T:TCC,"                  "22:17008073:T:TG,"

In [ ]:
OR11H1 22 15528177 22:15528177:C:T,22:15528363:C:T,22:15528467:G:GA,22:15528475:AC:A,22:15528532:C:A,22:15528649:G:A,22:15528650:G:A,22:15528668:G:A,22:15528753:C:T,22:15528777:G:GC,22:15528856:AT:A,22:15528926:T:A,22:15529137:T:A
GAB4 22 16962748 22:16962748:CCT:C,22:16963776:CGGCGGGG:C,22:16964805:G:GT,22:16966131:AG:A,22:16966169:G:A,22:16966249:TG:T,22:16970027:CT:C,22:16987991:G:A,22:16987996:G:T,22:16988055:C:CG,22:16988084:G:A,22:16988117:CTGGAGAAGAGCAGAGGCCG:C,22:16991922:C:T,22:16992153:GATAAACCA:G,22:16992162:GC:G,22:17007961:CG:C,22:17007961:C:CG,22:17007994:GCACGTGGC:G,22:17008009:T:TCC,22:17008073:T:TG